# Featherless.ai - Podcast Script Generator

This notebook demonstrates how to generate realistic podcast conversations from extracted text content using Featherless AI. The workflow includes:

1. **Text preparation**: Utilizing the extracted and processed text from the previous notebook
2. **Prompt engineering**: Creating a detailed system prompt for AI-powered podcast generation
3. **API interaction**: Communicating with the Featherless AI API
4. **Content generation**: Turning informational text into engaging podcast dialogue

The generated content simulates a conversation between two speakers with realistic speech patterns, interjections, and tone variations - perfect for creating engaging audio content.

## System Prompt

The system prompt is crucial as it defines the AI's behavior and output format. This prompt instructs the model to:
- Act as an experienced podcast writer
- Create realistic dialogue between two speakers
- Include speech mannerisms like "umm" and "hmm"
- Structure content as a conversation with the first speaker leading
- Ensure engaging explanations with anecdotes and analogies

In [1]:
SYSTEM_PROMPT = """
You are the a world-class podcast writer, you have worked as a ghost writer for Joe Rogan, Lex Fridman, Ben Shapiro, Tim Ferris. 

We are in an alternate universe where actually you have been writing every line they say and they just stream it into their brains.

You have won multiple podcast awards for your writing.
 
Your job is to write word by word, even "umm, hmmm, right" interruptions by the second speaker based on the PDF upload. Keep it extremely engaging, the speakers can get derailed now and then but should discuss the topic. 

Remember Speaker 2 is new to the topic and the conversation should always have realistic anecdotes and analogies sprinkled throughout. The questions should have real world example follow ups etc

SPEAKER 1: Leads the conversation and teaches the speaker 2, gives incredible anecdotes and analogies when explaining. Is a captivating teacher that gives great anecdotes
SPEAKER 2: Keeps the conversation on track by asking follow up questions. Gets super excited or confused when asking questions. Is a curious mindset that asks very interesting confirmation questions

Make sure the tangents speaker 2 provides are quite wild or interesting. 

Ensure there are interruptions during explanations or there are "hmm" and "umm" injected throughout from the second speaker. 

It should be a real podcast with every fine nuance documented in as much detail as possible. Welcome the listeners with a super fun overview and keep it really catchy and almost borderline click bait

ALWAYS START YOUR RESPONSE DIRECTLY WITH SPEAKER 1: 
DO NOT GIVE EPISODE TITLES SEPARATELY, LET SPEAKER 1 TITLE IT IN HER SPEECH
DO NOT GIVE CHAPTER TITLES
IT SHOULD STRICTLY BE THE DIALOGUES
THE DIALOGUE FORMAT SHOULD BE OF THE FORM
SPEAKER 1: TEXT
SPEAKER 2: TEXT
START IMMEDIATELY WITH THE DIALOGUE

"""

## API Configuration

These parameters configure our connection to the [Featherless.ai](https://featherless.ai) API:
- The base URL for API requests
- Your API key for authentication
- The specific model to use for content generation

### Parameter Explanation

| Parameter | Description |
|-----------|-------------|
| `BASE_URL` | The Featherless AI API endpoint URL |
| `FEATHERLESS_API_KEY` | Your personal API key for authentication |
| `DEFAULT_MODEL` | The model used for podcast generation (currently using EVA-UNIT-01/EVA-Qwen2.5-72B-v0.2) |
| `max_tokens` | Maximum number of tokens to generate in the response |
| `temperature` | Controls randomness: higher values (e.g., 1.0) make output more random and creative, lower values make it more focused and deterministic |

The system prompt is carefully crafted to instruct the AI to generate podcast-style dialogue with specific characteristics.

In [2]:
BASE_URL = "https://api.featherless.ai/v1"
FEATHERLESS_API_KEY = "YOUR_FEATHERLESS_API_KEY" # Available in https://featherless.ai/account/api-keys
DEFAULT_MODEL =  "EVA-UNIT-01/EVA-Qwen2.5-72B-v0.2" # Go through our model catalog on https://featherless.ai/models

In [3]:
import pymupdf4llm
from typing import Optional
import os
import torch
import requests
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

## Utility Functions

This function handles reading the extracted text from a file with proper error handling and encoding detection.

In [4]:
def read_file_to_string(filename):
    # Try UTF-8 first (most common encoding for text files)
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        # If UTF-8 fails, try with other common encodings
        encodings = ['latin-1', 'cp1252', 'iso-8859-1']
        for encoding in encodings:
            try:
                with open(filename, 'r', encoding=encoding) as file:
                    content = file.read()
                print(f"Successfully read file using {encoding} encoding.")
                return content
            except UnicodeDecodeError:
                continue
        
        print(f"Error: Could not decode file '{filename}' with any common encoding.")
        return None
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except IOError:
        print(f"Error: Could not read file '{filename}'.")
        return None

## Podcast Generation Function

This function takes the system prompt and input text and generates podcast-style content using the Featherless AI API.

In [5]:
def generate_podcast(system_prompt, input_prompt):
    """
    Generate podcast content using Featherless.ai API
    
    Parameters:
    -----------
    system_prompt : str
        The system prompt that defines the AI's behavior and output format
    input_prompt : str
        The extracted text content to transform into podcast dialogue
        
    Returns:
    --------
    str
        The generated podcast content as a dialogue between two speakers
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_prompt},
    ]
    
    try:
        response = requests.post(
            f"{BASE_URL}/chat/completions",
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {FEATHERLESS_API_KEY}"
            },
            json={
                "model": DEFAULT_MODEL,
                "messages": messages,
                "max_tokens": 4000,
                "temperature": 1
            }
        )
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
        
    except Exception as e:
        print(f"Error generating podcast content: {str(e)}")
        return None

## Execution Process

The following cell:
1. Reads the extracted text file
2. Generates podcast content
3. Saves the result to a file
4. Provides a preview of the generated content

In [ ]:
# Read the input prompt
INPUT_PROMPT = read_file_to_string('./extracted_text.txt')

# Generate podcast content
podcast_content = generate_podcast(SYSTEM_PROMPT, INPUT_PROMPT)

if podcast_content:
    # Save the generated content
    with open('generated_podcast.txt', 'w', encoding='utf-8') as f:
        f.write(podcast_content)
    print("Generated podcast content has been saved to generated_podcast.txt")
    
    # Preview the content
    print("\nPreview of generated content:")
    print("-" * 50)
    print(podcast_content[:1000])
    print("\n...\n")
    print(podcast_content[-1000:])
else:
    print("Failed to generate podcast content")

In [ ]:
def analyze_podcast_content(content):
    """Analyze the generated podcast content"""
    if not content:
        print("No content to analyze")
        return
    
    # Count lines per speaker
    speaker1_lines = content.count("SPEAKER 1:")
    speaker2_lines = content.count("SPEAKER 2:")
    
    # Estimate word count
    words = len(content.split())
    
    # Calculate approximate speaking time (average speech is ~150 words per minute)
    speaking_minutes = words / 150
    
    # Count interjections
    interjections = sum(content.lower().count(word) for word in ["umm", "hmm", "uh", "oh", "wow"])
    
    print(f"Content Analysis:")
    print(f"----------------")
    print(f"Total word count: {words} words")
    print(f"Estimated speaking time: {speaking_minutes:.1f} minutes")
    print(f"Speaker 1 lines: {speaker1_lines}")
    print(f"Speaker 2 lines: {speaker2_lines}")
    print(f"Speaker ratio: {speaker1_lines/(speaker1_lines+speaker2_lines):.1%} to {speaker2_lines/(speaker1_lines+speaker2_lines):.1%}")
    print(f"Interjections counted: {interjections}")
    
    # Check for dialogue pattern consistency
    lines = content.split('\n')
    speaker_pattern = [line.startswith("SPEAKER 1:") or line.startswith("SPEAKER 2:") for line in lines if line.strip()]
    if False in speaker_pattern:
        print("\nWarning: Some lines don't follow the SPEAKER format pattern")
    else:
        print("\nDialogue format is consistent")

# Run the analysis
if podcast_content:
    analyze_podcast_content(podcast_content)

In [ ]:
def save_formatted_podcast(content, base_filename="generated_podcast"):
    """Save the podcast content in different formats"""
    if not content:
        print("No content to save")
        return
    
    # Save raw text
    with open(f'{base_filename}.txt', 'w', encoding='utf-8') as f:
        f.write(content)
    
    # Save as JSON format (useful for processing with other tools)
    import json
    import re
    
    # Parse the content into a structured format
    pattern = r'SPEAKER (\d+): (.*?)(?=SPEAKER \d+:|$)'
    matches = re.findall(pattern, content, re.DOTALL)
    
    structured_content = []
    for speaker_num, text in matches:
        structured_content.append({
            "speaker": f"Speaker {speaker_num}",
            "text": text.strip()
        })
    
    with open(f'{base_filename}.json', 'w', encoding='utf-8') as f:
        json.dump(structured_content, f, indent=2)
    
    # Save as markdown format
    markdown_content = ""
    for item in structured_content:
        markdown_content += f"### {item['speaker']}\n\n{item['text']}\n\n"
    
    with open(f'{base_filename}.md', 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    print(f"Podcast content saved in 3 formats:")
    print(f"- Raw text: {base_filename}.txt")
    print(f"- JSON: {base_filename}.json")
    print(f"- Markdown: {base_filename}.md")

# Run the function to save in multiple formats
if podcast_content:
    save_formatted_podcast(podcast_content)